In [1]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.7 MB/s eta 0:00:00


In [2]:
import argparse
import json
from pathlib import Path
import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator,
    EarlyStoppingCallback
)

import os
os.environ["WANDB_DISABLED"] = "true"


file_path = "/content/drive/My Drive/Dissertation/cuad_qa_dataset.json"


model_candidates = [
    # "Jasu/bert-finetuned-squad-legalbert",
    # "nlpaueb/legal-bert-base-uncased",
    # "atharvamundada99/bert-large-question-answering-finetuned-legal",
    "facebook/bart-base"
]

# ========================================================
# 1. Mount Google Drive
# ========================================================

from google.colab import drive
drive.mount('/content/drive')

#========================================================
# 2. Load dataset
# ========================================================


def read_json(file_path):
    # Assumes the JSON file is a list of examples.
    print("Loading dataset from:", file_path)

    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


def data_preparation(data):
    # We'll just take a small subset (5430) for demonstration
    raw_data = data["train"][:3430]


    # Convert dataset to a huggingface Dataset
    dataset = Dataset.from_list(raw_data)
    print(dataset)

    split_dataset = dataset.train_test_split(test_size=0.1, seed=42)

    dataset_dict = DatasetDict({
        "train": split_dataset["train"],    # 90%
        "validation": split_dataset["test"], # 10%
    })

    print("DataSet Structure: ", dataset_dict)
    print("Train size:", len(dataset_dict["train"]))
    print("Validation size:", len(dataset_dict["validation"]))


    return dataset_dict


def prepare_features(entries, tokenizer, max_length, doc_stride):
    """
    Tokenizes a batch of examples with the fast tokenizer.
    Splits contexts into overlapping chunks.
    Returns a dict with the tokenized inputs as well as computed start_positions and end_positions.
    """
    # Tokenize with truncation on context only
    encodings = tokenizer(
        entries["question"],
        entries["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Debug: Print tokenization keys and sample mapping
    print("Tokenization keys:", list(encodings.keys()))
    print("Sample mapping (first 3):", encodings.get("overflow_to_sample_mapping", [])[:3])

    sample_mapping = encodings.pop("overflow_to_sample_mapping")
    offset_mapping = encodings.pop("offset_mapping")

    start_positions = []
    end_positions = []

    # Loop over each chunk and add debug prints
    for i, offsets in enumerate(offset_mapping):
        encoding = encodings.encodings[i]
        sequence_ids = encoding.sequence_ids

        # print(f"\n--- Chunk {i} Debug ---")
        # print("Sequence IDs:", sequence_ids)
        # print("Offset mapping:", offsets)

        sample_index = sample_mapping[i]
        # Get the original context for this example.
        context = entries["context"][sample_index]

        # --- Print the context chunk text for debugging ---
        # context_token_indices = [j for j, sid in enumerate(sequence_ids) if sid == 1]
        # if context_token_indices:
        #     ctx_start = offsets[context_token_indices[0]][0]
        #     ctx_end = offsets[context_token_indices[-1]][1]
        #     context_chunk = context[ctx_start:ctx_end]
        #     print("Context chunk text:", context_chunk)
        # else:
        #     print("No context tokens found in this chunk.")

        answers = entries["answers"][sample_index]
        # print("Original question:", entries["question"][sample_index])
        # print("Original answer info:", answers)

        # If no answer is provided, label as unanswerable.
        if len(answers["answer_start"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
        else:
            answer_start = answers["answer_start"][0]
            answer_text = answers["text"][0]
            answer_end = answer_start + len(answer_text)

            token_start_index = 0
            while token_start_index < len(sequence_ids) and sequence_ids[token_start_index] != 1:
                token_start_index += 1
            token_end_index = len(sequence_ids) - 1
            while token_end_index >= 0 and sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # print("Token start index before check:", token_start_index)
            # print("Token end index before check:", token_end_index)
            # print("Answer span in context (char indices):", answer_start, answer_end)

            if not (offsets[token_start_index][0] <= answer_start and offsets[token_end_index][1] >= answer_end):
                # print("Answer not fully in this chunk.")
                start_positions.append(0)
                end_positions.append(0)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= answer_start:
                    token_start_index += 1
                start_pos = token_start_index - 1

                while token_end_index >= 0 and offsets[token_end_index][1] >= answer_end:
                    token_end_index -= 1
                end_pos = token_end_index + 1

                # print("Computed start_pos:", start_pos)
                # print("Computed end_pos:", end_pos)
                # # Print the text for the computed tokens using the context and offsets.
                # token_start_text = context[offsets[start_pos][0]:offsets[start_pos][1]]
                # token_end_text = context[offsets[end_pos][0]:offsets[end_pos][1]]
                # print(f"Token {start_pos} text: '{token_start_text}'")
                # print(f"Token {end_pos} text: '{token_end_text}'")

                start_positions.append(start_pos)
                end_positions.append(end_pos)

    encodings["start_positions"] = start_positions
    encodings["end_positions"] = end_positions

    return encodings



def main():
    raw_data = read_json(file_path)
    data = data_preparation(raw_data)

    for model_name in model_candidates:
        print(f"\n==== Training model: {model_name} ====\n")


        # Create a safe folder name from the model name (replace "/" with "_")
        safe_model_name = model_name.replace("/", "_")
        output_dir = f"/content/drive/My Drive/Dissertation/model_output/{safe_model_name}"
        os.makedirs(output_dir, exist_ok=True)

        # Load tokenizer and model.
        tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        model = AutoModelForQuestionAnswering.from_pretrained(model_name)

        # Preprocess the dataset: tokenize and chunk.
        tokenized_train_dataset = data['train'].map(
            lambda examples: prepare_features(examples, tokenizer, 512, 128),
            batched=True,
            remove_columns=data['train'].column_names,
        )

                # Assuming your dataset_dict has a "validation" split:
        tokenized_val_dataset = data['validation'].map(
            lambda examples: prepare_features(examples, tokenizer, 512, 128),
            batched=True,
            remove_columns=data['validation'].column_names,
        )

        # print("Tokenized dataset samples:")
        # for entry in tokenized_train_dataset.select(range(3)):
        #     print(entry)


        # Define training arguments.
        training_args = TrainingArguments(
            output_dir=output_dir,
            evaluation_strategy="steps",
            learning_rate=5e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=2,
            weight_decay=0.01,
            save_strategy="steps",
            logging_steps=500,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            save_total_limit=1,
            gradient_accumulation_steps=1,
            disable_tqdm=False,
            push_to_hub=False,
            fp16=True,
        )

        # Create the Trainer.
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_train_dataset,
            eval_dataset=tokenized_val_dataset,
            tokenizer=tokenizer,
            data_collator=default_data_collator,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
        )

        # Train and save the fine-tuned model.
        trainer.train()
        trainer.save_model(output_dir)
        print(f"Model {model_name} saved to {output_dir}")


if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading dataset from: /content/drive/My Drive/Dissertation/cuad_qa_dataset.json
Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 3430
})
DataSet Structure:  DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 3087
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 343
    })
})
Train size: 3087
Validation size: 343

==== Training model: facebook/bart-base ====



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Tokenization keys: ['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping']
Sample mapping (first 3): [0, 0, 0]


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-2-51ae3229bd7e>:238: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
500,0.234700,0.130327
1000,0.122000,0.166148
1500,0.136100,0.137227


/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:394: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:394: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning 

Model facebook/bart-base saved to /content/drive/My Drive/Dissertation/model_output/facebook_bart-base


In [3]:
from google.colab import files
files.download('/content/drive/My Drive/Colab Notebooks/main_new.ipynb')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>